# Vectorized version of the slide 3.4 model

 - It is using the I inelastic selector to iterate
   during the return mapping over all the material
   points simultaneously
 - The gradients of potential functions deliver
   matrices with zero elements - `lambdify` delivers
   inhomogeneous arrays then. Quick fix was using the
   ZERO and ONE variables to include dummy arrays.
   But this produces many zeros for all material points.
   Probably, this indicates the limit of the concept.
   However, for prototyping this is still fine.
 - Would it be possible to use `theano`? It has operators
   for gradients and can run on Tensors. Check it here:
   [theano](https://theano-pymc.readthedocs.io/en/latest/tutorial)


In [39]:
%matplotlib widget
from bmcs_matmod.slide.vslide_34 import Slide34
import bmcs_matmod.slide.vslide_34 as slide_34
import numpy as np
import matplotlib.pylab as plt

In [40]:
material_params = dict(
    E_T=1, gamma_T=100, K_T=0, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=0.5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)

# Verify Slide34

In [41]:
slide = Slide34(**material_params)

To extract the names of the sympy symbols that
can be lambdified or used in a code generation to produce
C code a subclass Cymbol of sp.Symbol is defined
that adds another parameter called codename to be used
in lambdification. Any general material model
contains the property/attribute `state_var_shape`
specifying the key-value pair of the state variable
name and its shape to be allocated.

In [42]:
slide.state_var_shapes

{'w_pi': (),
 's_pi_x': (),
 's_pi_y': (),
 'z': (),
 'alpha_x': (),
 'alpha_y': (),
 'omega_T': (),
 'omega_N': (),
 'sig_pi': (),
 'tau_pi_x': (),
 'tau_pi_y': (),
 'Z': (),
 'X_x': (),
 'X_y': (),
 'Y_T': (),
 'Y_N': ()}

In [43]:
discr_shape = (1,)

state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}

In [44]:
state

{'w_pi': array([0.]),
 's_pi_x': array([0.]),
 's_pi_y': array([0.]),
 'z': array([0.]),
 'alpha_x': array([0.]),
 'alpha_y': array([0.]),
 'omega_T': array([0.]),
 'omega_N': array([0.]),
 'sig_pi': array([0.]),
 'tau_pi_x': array([0.]),
 'tau_pi_y': array([0.]),
 'Z': array([0.]),
 'X_x': array([0.]),
 'X_y': array([0.]),
 'Y_T': array([0.]),
 'Y_N': array([0.])}

In [45]:
eps_Ema = np.zeros(discr_shape+(3,), dtype=np.float_)

In [46]:
w_n1, s_x_n1, s_y_n1 = np.einsum('...a->a...',eps_Ema)

Eps_n = np.array([state[eps_name] for eps_name in slide.Eps_names])
Eps_k = np.copy(Eps_n)
Sig_k = np.array([state[sig_name] for sig_name in slide.Sig_names])

In [47]:
s_x_n1.shape

(1,)

In [48]:
Eps_k.shape
Sig_k.shape

(8, 1)

In [49]:
slide.symb.get_Sig_(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k)[0].shape

(8, 1)

In [50]:
slide.symb.Sig_

Matrix([[-E_N*(-2*w + 2*w^{\pi})*(-omega_N*Piecewise((0, \sigma^\pi <= 0), (1, True))/2 + 1/2), -E_T*(1 - omega_T)*(s^{\pi}_x - s_x), -E_T*(1 - omega_T)*(s^{\pi}_y - s_y), K_T*z, alpha_x*gamma_T, alpha_y*gamma_T, E_T*(-s^{\pi}_x + s_x)**2/2 + E_T*(-s^{\pi}_y + s_y)**2/2, E_N*(w - w^{\pi})**2*Piecewise((0, \sigma^\pi <= 0), (1, True))/2]])

In [51]:
slide.symb.dSig_dEps_

Matrix([
[ E_N*I*(omega_N*Piecewise((O, O >= \sigma^\pi), (1, True)) - 1),                     I*O,                     I*O,   I*O,       I*O,       I*O,                     I*O, E_N*I*(-2*w + 2*w^{\pi})*Piecewise((O, O >= \sigma^\pi), (1, True))/2],
[                                                            I*O,     E_T*I*(omega_T - 1),                     I*O,   I*O,       I*O,       I*O, E_T*I*(s^{\pi}_x - s_x),                                                                   I*O],
[                                                            I*O,                     I*O,     E_T*I*(omega_T - 1),   I*O,       I*O,       I*O, E_T*I*(s^{\pi}_y - s_y),                                                                   I*O],
[                                                            I*O,                     I*O,                     I*O, I*K_T,       I*O,       I*O,                     I*O,                                                                   I*O],
[                      

In [52]:
ONES = np.ones_like(s_x_n1)
ZEROS = np.zeros_like(s_x_n1)

In [53]:
dSig_dEps_k = slide.symb.get_dSig_dEps_(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k, ZEROS, ONES)
dSig_dEps_k.shape

(8, 8, 1)

In [54]:
ix1, ix2 = np.ix_((0,1,2),(0,1,2))

In [55]:
dSig_dEps_k[ix1,ix2,0]

array([[-1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0., -1.]])

In [56]:
H_sig_pi = slide.symb.get_H_sig_pi_(Sig_k)
H_sig_pi.shape

(1,)

In [57]:
f_k = slide.symb.get_f_(Eps_k, Sig_k, H_sig_pi)
f_k.shape

(1,)

In [58]:
%%capture
df_dSig_k = slide.symb.get_df_dSig_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)

In [59]:
df_dSig_k.shape

(8, 1, 1)

In [60]:
%%capture
ddf_dEps_k = slide.symb.get_ddf_dEps_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)

In [61]:
ddf_dEps_k.shape

(8, 1, 1)

In [62]:
df_dEps_k = np.einsum(
    'ik...,ji...->jk...', df_dSig_k, dSig_dEps_k) + ddf_dEps_k
df_dEps_k.shape

(8, 1, 1)

In [63]:
%%capture
slide.get_f_df(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k)

In [64]:
slide.get_Eps_k1(w_n1, s_x_n1, s_y_n1, Eps_n, ONES, Sig_k, Eps_k)

array([[0.99],
       [0.  ],
       [0.  ],
       [1.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ]])

# Compare the single point and multiple point versions

In [65]:
%matplotlib widget
from bmcs_matmod.slide.vslide_34 import Slide34
import bmcs_matmod.slide.vslide_34 as slide_34
import numpy as np
import matplotlib.pylab as plt

In [66]:
material_params = dict(
    E_T=1, gamma_T=1, K_T=0, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=0.5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)

In [67]:
slide = Slide34(**material_params)

In [68]:
discr_shape = (1,)
state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}
eps_Ema = np.zeros(discr_shape+(3,), dtype=np.float_) + 1e-9
eps_Ema.shape

(1, 3)

In [69]:
s_max = 4
eps_Ema[0,1] = s_max
slide.k_max = 3
sig_Ema, D_Emab = slide.get_corr_pred(eps_Ema, 1, **state)

Compare the values in the two versions

In [70]:
sig_Ema, state

(array([[-4.99974901e-03,  7.00144994e-01,  1.75036248e-10]]),
 {'w_pi': array([0.00499975]),
  's_pi_x': array([0.499975]),
  's_pi_y': array([1.2499375e-10]),
  'z': array([0.499975]),
  'alpha_x': array([0.499975]),
  'alpha_y': array([1.2499375e-10]),
  'omega_T': array([0.79996]),
  'omega_N': array([4.99975001e-19]),
  'sig_pi': array([-0.00499975]),
  'tau_pi_x': array([0.70014499]),
  'tau_pi_y': array([1.75036248e-10]),
  'Z': array([0.]),
  'X_x': array([0.499975]),
  'X_y': array([1.2499375e-10]),
  'Y_T': array([6.1250875]),
  'Y_N': array([1.24987451e-05])})

In [71]:
from bmcs_matmod.slide.slide_32 import Slide32
Sig_k_32 = np.zeros((8,))
Eps_k_32 = np.zeros((8,)) + 1e-9
slide32 = Slide32(**material_params)
Eps_k_32, Sig_k_32, k = slide32.get_sig_n1(s_max,0,0,Sig_k_32, Eps_k_32,3)

In [72]:
Eps_k_32

array([4.99975001e-01, 7.50012500e-10, 4.99975100e-03, 4.99975001e-01,
       4.99975001e-01, 7.50012500e-10, 7.99960000e-01, 1.00000000e-09])

In [73]:
Sig_k_32

array([ 7.00145001e-01, -1.50032501e-10, -4.99975100e-03,  0.00000000e+00,
        4.99975001e-01,  7.50012500e-10,  6.12508750e+00,  0.00000000e+00])

# Plotting of the elementary monotonic loading

Plot the stress strain curve and evaluate the energy dissipation evolution

In [74]:
material_params = dict(
    E_T=1000, gamma_T=10, K_T=0, S_T=0.001, c_T=1, bartau=3, 
    E_N=100, S_N=0.01, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)

# Prototyping a method for plotting of the stress strain curve

In [93]:
slide = Slide34(**material_params)
discr_shape = (1,)
state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}
n_s = 1000
idx = 0
s_max = .2
s_range = np.linspace(0,s_max,n_s)
tau_range = []
slide.k_max = 20
eps_Ema = np.zeros(discr_shape+(3,), dtype=np.float_) + 1e-9
for i, s in enumerate(s_range):
    eps_Ema[0,idx] = s
    sig_Ema, D_Emab = slide.get_corr_pred(eps_Ema, 1, **state)
    tau_range.append(sig_Ema[0,idx])

In [94]:
_, ax = plt.subplots(1,1)
ax.plot(s_range, tau_range)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
slide.interact()